In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer, util
from gliner import GLiNER

## Modelo Analisis de sentimientos

In [11]:
dataset_sentimientos = pd.read_csv('./databases/sentimientos.csv')

In [12]:
dataset_sentimientos.head()

,clase,frase
0,0,me siento un poco nostalgico hoy.
1,0,"todo parece tan gris, como si algo faltara."
2,0,"no tengo ganas de hacer nada, estoy algo decaido."
3,0,echo de menos esos tiempos felices que ya no v...
4,0,"me invade una sensacion de tristeza, sin motiv..."


In [14]:
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1') 

labels = [(0, "Melancolico"), (1, "Ni fu Ni fa"), (2, "Alegre")]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(dataset_sentimientos['frase'].to_list(), dataset_sentimientos['clase'].to_list(), test_size=0.2, shuffle=True,random_state=42)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
X_train_vectorized = model.encode(X_train)
X_test_vectorized = model.encode(X_test)
# Creación y entrenamiento del modelo de Regresión Logística Multinomial
modelo_sentimientos = LogisticRegression(max_iter=100, solver='lbfgs')
modelo_sentimientos.fit(X_train_vectorized, y_train)

# Evaluación del modelo de Regresión Logística
y_pred_sentimientos = modelo_sentimientos.predict(X_test_vectorized)
acc_sentimientos = accuracy_score(y_test, y_pred_sentimientos)
report_sentimientos = classification_report(y_test, y_pred_sentimientos, zero_division=1)

print("Precisión Regresión Logística:", acc_sentimientos)
print("Reporte de clasificación Regresión Logística:\n", report_sentimientos)


Precisión Regresión Logística: 0.9393939393939394
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       0.91      0.91      0.91        11
           1       0.91      0.91      0.91        11
           2       1.00      1.00      1.00        11

    accuracy                           0.94        33
   macro avg       0.94      0.94      0.94        33
weighted avg       0.94      0.94      0.94        33



In [ ]:
for i in range(len(y_test)):
    if y_test[i] != y_pred_sentimientos[i]:
        print(f"Diferencia encontrada en el índice {i}:")
        print(f"X_test: {X_test[i]}")
        print(f"y_test: {y_test[i]}")
        print(f"y_pred_sentimientos: {y_pred_sentimientos[i]}")
        print()  # Línea en blanco para separar las salidas

Diferencia encontrada en el índice 8:
X_test: me siento como si el dia hubiera pasado desapercibido, sin nada memorable.
y_test: 1
y_pred_sentimientos: 0

Diferencia encontrada en el índice 12:
X_test: no encuentro consuelo en nada, todo parece sombrio.
y_test: 0
y_pred_sentimientos: 1



In [ ]:
# Make something to export the model to another notebook
import joblib

joblib.dump(modelo_sentimientos, './models/modelo_sentimientos.pkl')

print("Modelo exportado correctamente")

Modelo exportado correctamente


## Modelo Recomendaciones

In [47]:
juegos_dataset = pd.read_csv('./databases/juegos_procesados.csv')
libros_dataset = pd.read_csv('./databases/libros_procesados.csv')
peliculas_dataset = pd.read_csv('./databases/peliculas_procesados.csv')

In [15]:
modelo_recomendaciones = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
modelo_gliner = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")

In [48]:
juegos_embeddings = modelo_recomendaciones.encode(juegos_dataset['description'].tolist(),convert_to_tensor=True)
libros_embeddings = modelo_recomendaciones.encode(libros_dataset['description'].tolist(),convert_to_tensor=True)
peliculas_embeddings = modelo_recomendaciones.encode(peliculas_dataset['description'].tolist(),convert_to_tensor=True)

In [30]:
consulta_usuario = 'Historia de un hombre que vive en la selva como Tarzan'
labels = ['Name','Title','City']
entidades_consulta = modelo_gliner.predict_entities(consulta_usuario,labels,threshold=0.5)
entidades_consulta
#print(f"Entidades encontradas en la consulta: {entidades_consulta}")
print(f"Entidades encontradas en la consulta: {entidades_consulta[0]['label']}")

Entidades encontradas en la consulta: Name


In [51]:
user_embedding = modelo_recomendaciones.encode('Historia de un hombre que vive en la selva',convert_to_tensor=True)

coseno = util.cos_sim(user_embedding, libros_embeddings)[0]
index = coseno.argsort(descending=True)[:3].tolist()
# Extraer las recomendaciones
recomendaciones = [(libros_dataset['title'].iloc[i], coseno[i].item()) for i in index]
recomendaciones


[('The Jungle Book', 0.6341819763183594),
 ('The Beast in the Jungle', 0.6314946413040161),
 ('The Jungle', 0.6167158484458923)]

In [ ]:
joblib.dump(modelo_recomendaciones, './models/modelo_recomendaciones.pkl')

print("Modelo exportado correctamente")

Modelo exportado correctamente
